# Souce
* https://towardsdatascience.com/boosting-showdown-scikit-learn-vs-xgboost-vs-lightgbm-vs-catboost-in-sentiment-classification-f7c7f46fd956

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
cd drive/My\ Drive/Colab\ Notebooks/voice_phishing/

/content/drive/My Drive/Colab Notebooks/voice_phishing


# Import librairie

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 57 kB/s 


In [ ]:
# Importing packages and data
from time import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix


# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import (RandomForestClassifier,
                              AdaBoostClassifier,
                              GradientBoostingClassifier,
                              HistGradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import seaborn as sns

# Import and prepare the data

In [ ]:
# df_train = pd.read_csv('train.csv').sample(frac=1.0)
# df_test = pd.read_csv('test.csv').sample(frac=1.0)

# use this data sets incase we want the ration of 8:2
df_train = pd.read_csv('train_test_sets/train_KorCCVi_v2_fullcleansed.csv')
df_test = pd.read_csv('train_test_sets/test_KorCCVi_v2_fullcleansed.csv')

# import the full dataset
# df_dataset = pd.read_csv('KorCCVi_v2_fullcleansed.csv').sample(frac=1.0)

In [ ]:
# df_dataset

In [ ]:
# df_train

In [ ]:
# df_test

In [ ]:
# df_test.head()

# train_test_split

In [ ]:
# data, labels = df_dataset['transcript'], df_dataset['label']

# # numericalization of the text into numbers
# data = tfidf.fit_transform(data).toarray()
# X_test = tfidf.transform(df_test['transcript']).toarray()

# # Split the data in train and test set, reseve 10% of the for the test set
# X_train, X_test, y_train, y_test = train_test_split(
#     data,
#     labels,
#     test_size=0.1,
#     random_state=42, 
#     shuffle= True
# )

# # slip the train set in train and eval, reseve 10% of the data for thse evaluation
# X_train_sub, X_val, y_train_sub, y_val = train_test_split(
#     X_train,
#     y_train, 
#     test_size=0.1,
#     random_state=42, 
#     shuffle= True
# )

In [ ]:
# # check the shape of each set
# print("X_train shape: {}".format(X_train.shape))
# print("y_train shape: {}".format(y_train.shape))

# print("X_train_sub shape: {}".format(X_train_sub.shape))
# print("y_train_sub shape: {}".format(y_train_sub.shape))

# print("X_val shape: {}".format(X_val.shape))
# print("y_val shape: {}".format(y_val.shape))

# print("X_test shape: {}".format(X_test.shape))
# print("y_test shape: {}".format(y_test.shape))

In [ ]:
%%time
# Preprocessing our data
# Convert a collection of raw documents to a matrix of TF-IDF features.
tfidf = TfidfVectorizer(# analyzer='word', # default='word'
                        ngram_range=(1, 3), #(2, 6), # 3,6
                        # max_df=1, 
                        min_df=2, # 10
                        max_features=2000, #300, 500, 10000 (thsi will limit the vocavulary) 
                        # sublinear_tf=True,
                        use_idf=True
                        )

CPU times: user 38 µs, sys: 6 µs, total: 44 µs
Wall time: 50.5 µs


In [ ]:
tfidf

TfidfVectorizer(max_features=2000, min_df=2, ngram_range=(1, 3))

In [ ]:
# numericalization of the text into numbers
X_train = tfidf.fit_transform(df_train['transcript']).toarray()
X_test = tfidf.transform(df_test['transcript']).toarray()

y_train, y_test = df_train['label'], df_test['label']

# slip the train set and reseve 20% of the data for the evaluation
X_train_sub, X_valid, y_train_sub, y_valid = train_test_split(X_train, 
                                                              y_train, 
                                                              test_size=0.2,
                                                              random_state=42, 
                                                              shuffle= True
                                                              )

# training with fulldataset
# y_train = df_train['label']
# X_train_sub, X_test, y_train_sub, y_test = train_test_split(X_train, y_train, test_size=0.3, shuffle= True)

In [ ]:
# check the shape of each set
print("X_train shape: {}".format(X_train_sub.shape))
print("X_val shape: {}".format(X_valid.shape))
print("X_test shape: {}".format(X_test.shape))

print("y_train shape: {}".format(y_train_sub.shape))
print("y val shape: {}".format(y_valid.shape))
print("y_test shape: {}".format(y_test.shape))

X_train shape: (1872, 2000)
X_val shape: (469, 2000)
X_test shape: (586, 2000)
y_train shape: (1872,)
y val shape: (469,)
y_test shape: (586,)


In [ ]:
print(X_train_sub)

[[0.01190997 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.01011084 0.         0.        ]
 [0.         0.         0.         ... 0.03720847 0.         0.        ]
 ...
 [0.         0.         0.         ... 0.03197549 0.         0.        ]
 [0.         0.         0.         ... 0.16609568 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
X_test

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01638131, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00858095, 0.        ,
        0.        ],
       [0.01272806, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
tfidf.get_feature_names_out()

array(['10', '11', '12', ..., '힘들', '힘들 어서', '힘들 어요'], dtype=object)

In [ ]:
# Setting up our results dataframe
df_results = pd.DataFrame(columns=['F1_score', 'Precision', 'Recall', 'Accuracy', 'Accuracy1', 'Training time', 'Test time'])

# Build the models


In [ ]:
#llogistic regression
lr = LogisticRegression(
    # C=10,
    # random_state=1234
)
lr

LogisticRegression()

In [ ]:
#Decision Tree
dt = DecisionTreeClassifier(
    # max_depth=10, 
    # random_state=1234
)
dt

DecisionTreeClassifier()

In [ ]:
#Random Forest
rf = RandomForestClassifier(
    # n_estimators=100,
    # max_depth=20,
    # max_features=0.06,
    # n_jobs=6,
    # random_state=1234
)
rf

RandomForestClassifier()

In [ ]:
#Adaboost
# base_estim = DecisionTreeClassifier(max_depth=1, max_features=0.06)                            
ab = AdaBoostClassifier(
    # base_estimator=base_estim,
    # n_estimators=500,
    # learning_rate=0.5,
    # random_state=1234
)
ab

AdaBoostClassifier()

In [ ]:
#Gradient Boosted Decision Trees (Scikit-Learn)
gbm = GradientBoostingClassifier(
    # n_estimators=2000,
    # subsample=0.67,
    # max_features=0.06,
    # validation_fraction=0.1,
    # n_iter_no_change=15,
    # verbose=0,
    # random_state=1234
)
gbm

GradientBoostingClassifier()

In [ ]:
#XGBoost
xgb = XGBClassifier(
    # n_estimators=2000,
    # tree_method='hist',
    # subsample=0.67,
    # colsample_level=0.06,
    # verbose=0,
    # n_jobs=6,
    # random_state=1234
)
xgb

XGBClassifier()

In [ ]:
#LightGBM
lgbm = LGBMClassifier(
    # n_estimators=2000,
    # feature_fraction=0.06,
    # bagging_fraction=0.67,
    # bagging_freq=1,
    # verbose=0,
    # n_jobs=6,
    # random_state=1234
)
lgbm

LGBMClassifier()

In [ ]:
#CatBoost
cb = CatBoostClassifier(
    # n_estimators=2000,
    # colsample_bylevel=0.06,
    # max_leaves=31,
    # subsample=0.67,
    # thread_count=6,
    # verbose=0,
    # random_state=1234
)
cb

In [ ]:
cb.get_params()

{}

In [ ]:
#Bonus: Histogram-based Gradient Boosting Machine (Scikit-Learn)
hgbm = HistGradientBoostingClassifier(
    # max_iter=2000,
    # validation_fraction=0.1,
    # n_iter_no_change=15,
    # verbose=0,
    # random_state=1234
)
hgbm

HistGradientBoostingClassifier()

In [ ]:
#Comparing our Competitors
models = [lr, dt, rf, ab, gbm, hgbm, xgb, lgbm, cb]
# models = [rf, xgb, lgbm, cb]
model_names = [i.__class__.__name__ for i in models]

In [ ]:
#to specify which of our models require early stopping within the .fit() method.
es_models = [
             'XGBClassifier',
             'LGBMClassifier',
             'CatBoostClassifier'
]

# train models

In [ ]:
from timeit import default_timer as timer

start = timer()
#Train each of our candidate models
for m, n in zip(models, model_names):
  train_time = 0
  test_time = 0
    
  if n in es_models:
      start_train = time()
      m.fit(X_train_sub,
            y_train_sub,
            eval_set = [(X_valid, y_valid)],
            early_stopping_rounds=15,
            verbose=0)
      train_time = time() - start_train
  else:
      start_train = time()
      m.fit(X_train, y_train)
      train_time = time() - start_train

  # Make test of the model
  start_test = time()
  predicted = m.predict(X_test)
  test_time = time() - start_test

  print('-'*80)
  print(n)
  accuracy = np.mean(predicted == y_test)
  print(accuracy)

  # Use score method to get accuracy of model
  score = m.score(X_test, y_test)
  print(score)

  print(classification_report(y_test, predicted))

  # cm = confusion_matrix(y_test, predicted)
  # print(cm)
  print('#'*80)

  df_results.loc[n] = [None, None, None, None, accuracy, train_time, test_time]
  
  del m
    
train_test_time = timer() - start
print('The time is {:.4f} seconds'.format(train_test_time))

--------------------------------------------------------------------------------
LogisticRegression
1.0
1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       448
           1       1.00      1.00      1.00       138

    accuracy                           1.00       586
   macro avg       1.00      1.00      1.00       586
weighted avg       1.00      1.00      1.00       586

################################################################################
--------------------------------------------------------------------------------
DecisionTreeClassifier
0.9897610921501706
0.9897610921501706
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       448
           1       0.97      0.99      0.98       138

    accuracy                           0.99       586
   macro avg       0.98      0.99      0.99       586
weighted avg       0.99      0.99      0.99       586

###############

In [ ]:
# cb.get_all_params()

In [ ]:
df_results

,F1_score,Precision,Recall,Accuracy,Accuracy1,Training time,Test time
LogisticRegression,NaN,NaN,NaN,NaN,1.000000,0.161276,0.002261
DecisionTreeClassifier,NaN,NaN,NaN,NaN,0.989761,0.838473,0.006275
RandomForestClassifier,NaN,NaN,NaN,NaN,1.000000,1.646268,0.031493
AdaBoostClassifier,NaN,NaN,NaN,NaN,1.000000,8.285576,0.156767
GradientBoostingClassifier,NaN,NaN,NaN,NaN,1.000000,34.827289,0.006557
HistGradientBoostingClassifier,NaN,NaN,NaN,NaN,1.000000,19.715119,0.020977
XGBClassifier,NaN,NaN,NaN,NaN,0.988055,2.309119,0.019061
LGBMClassifier,NaN,NaN,NaN,NaN,1.000000,10.514986,0.010745
CatBoostClassifier,NaN,NaN,NaN,NaN,1.000000,102.418896,0.341318


In [ ]:
## Testing LinearSVC
start = timer()

svc = LinearSVC(
    # C = 10
)
# clf = svm.SVC(kernel='linear', C=10)
svc

LinearSVC()

In [ ]:
start_time = time()
svc.fit(X_train, y_train)
run_time = time() - start_time

start_test = time()
preds = svc.predict(X_test)
test_time = time() - start_test

accuracy = np.mean(preds == y_test)
df_results.loc['LinearSVC'] = [None, None, None, None, accuracy, run_time, test_time]

train_time = timer() - start
print('The time is {:.4f} seconds'.format(train_time))

The time is 0.0723 seconds


In [ ]:
df_results

,F1_score,Precision,Recall,Accuracy,Accuracy1,Training time,Test time
LogisticRegression,NaN,NaN,NaN,NaN,1.000000,0.161276,0.002261
DecisionTreeClassifier,NaN,NaN,NaN,NaN,0.989761,0.838473,0.006275
RandomForestClassifier,NaN,NaN,NaN,NaN,1.000000,1.646268,0.031493
AdaBoostClassifier,NaN,NaN,NaN,NaN,1.000000,8.285576,0.156767
GradientBoostingClassifier,NaN,NaN,NaN,NaN,1.000000,34.827289,0.006557
HistGradientBoostingClassifier,NaN,NaN,NaN,NaN,1.000000,19.715119,0.020977
XGBClassifier,NaN,NaN,NaN,NaN,0.988055,2.309119,0.019061
LGBMClassifier,NaN,NaN,NaN,NaN,1.000000,10.514986,0.010745
CatBoostClassifier,NaN,NaN,NaN,NaN,1.000000,102.418896,0.341318
LinearSVC,NaN,NaN,NaN,NaN,1.000000,0.048701,0.002273


In [ ]:
# table = df_results[['accuracy','run_time']].head()

In [ ]:
# ax = df_results.plot()
# fig = ax.get_figure()
# fig.savefig('asdf.png')

In [ ]:
# ls -l

In [ ]:
# example of grid searching key hyperparameters for adaboost on a classification dataset
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import AdaBoostClassifier


# # define the model with default hyperparameters
# model = AdaBoostClassifier()

# define the grid of values to search
grid = dict()
grid['n_estimators'] = [10, 50, 100, 500]
grid['learning_rate'] = [0.0001, 0.001, 0.01, 0.1, 1.0]

# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# define the grid search procedure
grid_search = GridSearchCV(estimator=ab, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy')

# execute the grid search
grid_result = grid_search.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 1.000000 using {'learning_rate': 1.0, 'n_estimators': 500}
0.956719 (0.009821) with: {'learning_rate': 0.0001, 'n_estimators': 10}
0.956719 (0.009821) with: {'learning_rate': 0.0001, 'n_estimators': 50}
0.956719 (0.009821) with: {'learning_rate': 0.0001, 'n_estimators': 100}
0.956719 (0.009821) with: {'learning_rate': 0.0001, 'n_estimators': 500}
0.956719 (0.009821) with: {'learning_rate': 0.001, 'n_estimators': 10}
0.956719 (0.009821) with: {'learning_rate': 0.001, 'n_estimators': 50}
0.956719 (0.009821) with: {'learning_rate': 0.001, 'n_estimators': 100}
0.963267 (0.010997) with: {'learning_rate': 0.001, 'n_estimators': 500}
0.956719 (0.009821) with: {'learning_rate': 0.01, 'n_estimators': 10}
0.962839 (0.011301) with: {'learning_rate': 0.01, 'n_estimators': 50}
0.968391 (0.009199) with: {'learning_rate': 0.01, 'n_estimators': 100}
0.996441 (0.003834) with: {'learning_rate': 0.01, 'n_estimators': 500}
0.973803 (0.008510) with: {'learning_rate': 0.1, 'n_estimators': 10}
0.996583

# Model with recall and precision metrics


## librairies

In [ ]:
!pip install psutil nodejs matplotlib seaborn==0.8.1 neptune-client==0.3.7 neptune-contrib[versioning]==0.11.0 neptune-notebooks==0.0.8 scikit-plot

     |████████████████████████████████| 178 kB 4.8 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 829 kB 34.2 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 181 kB 52.5 MB/s 
     |████████████████████████████████| 276 kB 49.0 MB/s 
     |████████████████████████████████| 2.0 MB 41.4 MB/s 
     |████████████████████████████████| 132 kB 60.8 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 8.7 MB 34.3 MB/s 
     |████████████████████████████████| 127 kB 65.9 MB/s 
     |████████████████████████████████| 130 kB 51.6 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 64 kB 2.0 MB/s 
  Created wheel for seaborn: filename=seaborn-0.8.1-py3-none-any.whl size=184850 sha256=22c0fe0e1d974e443e797b7c8c765b4fa5e984360da6dea4a6269c6

## Utils.py

In [ ]:
import matplotlib.pyplot as plt
import neptune
from neptunecontrib.monitoring.utils import send_figure
import pandas as pd
import scikitplot.metrics as plt_metrics
from scikitplot.helpers import binary_ks_curve
import seaborn as sns
import sklearn.metrics as sk_metrics

plt.rcParams.update({'font.size': 18})
plt.rcParams.update({'figure.figsize': [16, 12]})


def log_binary_classification_metrics(y_true, y_pred, threshold=0.5):
    # Confusion Matrix
    # fig, ax = plt.subplots()
    # plot_confusion_matrix(y_true, y_pred[:, 1] > threshold, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    # plt.close()

    # Plot classification report
    # fig = plot_classification_report(y_true, y_pred[:, 1] > threshold)
    # send_figure(fig, channel_name='metric_charts')
    # plt.close()

    scores = _class_metrics(y_true, y_pred[:, 1], threshold)
    # for metric_name, score in scores.items():
        # print('{} :{}'.format(metric_name, score))
        # neptune.log_metric(metric_name, score)

    # ROC AUC
    roc_auc = sk_metrics.roc_auc_score(y_true, y_pred[:, 1])
    # print('ROC AUC score {}'.format(roc_auc))
    # neptune.log_metric('roc_auc', roc_auc)

    fig, ax = plt.subplots()
    plt_metrics.plot_roc(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # PR AUC
    avg_precision = sk_metrics.average_precision_score(y_true, y_pred[:, 1])
    # print('Average precision score {}'.format(avg_precision))
    # neptune.log_metric('avg_precision', avg_precision)

    fig, ax = plt.subplots()
    plt_metrics.plot_precision_recall(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # Brier loss
    brier = sk_metrics.brier_score_loss(y_true, y_pred[:, 1])
    # print('Brier loss {}'.format(brier))
    # neptune.log_metric('brier_loss', brier)

    # Log loss
    log_loss = sk_metrics.log_loss(y_true, y_pred)
    # print('Log loss {}'.format(log_loss))
    # neptune.log_metric('log_loss', log_loss)

    # Kolmogorov Smirnow
    res = binary_ks_curve(y_true, y_pred[:, 1])
    ks_stat = res[3]
    # print('KS stat {}'.format(ks_stat))
    # neptune.log_metric('ks_statistic', ks_stat)

    fig, ax = plt.subplots()
    plt_metrics.plot_ks_statistic(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # Cumulative gain
    fig, ax = plt.subplots()
    plt_metrics.plot_cumulative_gain(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # Lift curve
    fig, ax = plt.subplots()
    plt_metrics.plot_lift_curve(y_true, y_pred, ax=ax)
    # send_figure(fig, channel_name='metric_charts')
    plt.close()

    # Plots by threshold
    figs = plot_metrics_per_threshold(y_true, y_pred[:, 1])

    for fig in figs:
        # send_figure(fig, channel_name='metrics_by_threshold')
        plt.close()

    return scores


def plot_confusion_matrix(y_true, y_pred_class, ax=None):
    cmap = plt.get_cmap('Blues')
    cm = sk_metrics.confusion_matrix(y_true, y_pred_class)
    sns.heatmap(cm, cmap=cmap, annot=True, fmt='g', ax=ax)


def plot_metrics_per_threshold(y_true, y_pred_positive):
    scores_by_thres = _class_metrics_by_threshold(y_true, y_pred_positive)
    figs = []
    for name in scores_by_thres.columns:
        if name == 'threshold':
            continue
        else:
            best_thres, best_score = _get_best_thres(scores_by_thres, name)
            fig, ax = plt.subplots()
            ax.plot(scores_by_thres['threshold'], scores_by_thres[name])
            ax.set_title('{} by threshold'.format(name))
            ax.axvline(x=best_thres, color='red')
            ax.text(x=best_thres, y=0.96 * best_score,
                    s='thres={:.4f}\nscore={:.4f}'.format(best_thres, best_score),
                    color='red')
            figs.append(fig)
    return figs


def plot_classification_report(y_true, y_pred_class):
    report = sk_metrics.classification_report(y_true, y_pred_class, output_dict=True)
    report_df = pd.DataFrame(report).transpose().round(4)

    fig, ax = plt.subplots()
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=report_df.values,
             collabels=report_df.columns,
             rowlabels=report_df.index,
             loc='center',
             bbox=[0.2, 0.2, 0.8, 0.8])
    fig.tight_layout()

    return fig


def _class_metrics(y_true, y_pred, threshold):
    y_pred_class = y_pred > threshold

    tn, fp, fn, tp = sk_metrics.confusion_matrix(y_true, y_pred_class).ravel()

    true_positive_rate = tp / (tp + fn)
    true_negative_rate = tn / (tn + fp)
    positive_predictive_value = tp / (tp + fp)
    negative_predictive_value = tn / (tn + fn)
    false_positive_rate = fp / (fp + tn)
    false_negative_rate = fn / (tp + fn)
    false_discovery_rate = fp / (tp + fp)

    scores = {'accuracy': sk_metrics.accuracy_score(y_true, y_pred_class),
              'precision': sk_metrics.precision_score(y_true, y_pred_class),
              'recall': sk_metrics.recall_score(y_true, y_pred_class),
              'f1_score': sk_metrics.fbeta_score(y_true, y_pred_class, beta=1),
              'f2_score': sk_metrics.fbeta_score(y_true, y_pred_class, beta=2),
              'matthews_corrcoef': sk_metrics.matthews_corrcoef(y_true, y_pred_class),
              'cohen_kappa': sk_metrics.cohen_kappa_score(y_true, y_pred_class),
              'true_positive_rate': true_positive_rate,
              'true_negative_rate': true_negative_rate,
              'positive_predictive_value': positive_predictive_value,
              'negative_predictive_value': negative_predictive_value,
              'false_positive_rate': false_positive_rate,
              'false_negative_rate': false_negative_rate,
              'false_discovery_rate': false_discovery_rate,
              }

    return scores


def _class_metrics_by_threshold(y_true, y_pred_positive, thres_nr=100):
    thresholds = [i / thres_nr for i in range(1, thres_nr, 1)]

    scores_per_thres = []
    for thres in thresholds:
        scores = _class_metrics(y_true, y_pred_positive, thres)
        scores['threshold'] = thres
        scores_per_thres.append(pd.Series(scores))

    return pd.DataFrame(scores_per_thres)


def _get_best_thres(scores_by_thres, name):
    best_res = scores_by_thres[scores_by_thres[name] == scores_by_thres[name].max()][['threshold', name]]
    position = len(best_res) // 2
    result = best_res.iloc[position].to_dict()
    return result['threshold'], result[name]


## training 2

In [ ]:
import lightgbm
import neptune
from neptunecontrib.monitoring.utils import pickle_and_send_artifact
from neptunecontrib.versioning.data import log_data_version
import pandas as pd

# from utils import log_binary_classification_metrics

from timeit import default_timer as timer

start = timer()
#Train each of our candidate models
for m, n in zip(models, model_names):
  if n in es_models:
    start_time = time()
    m.fit(X_train_sub,
          y_train_sub,
          eval_set = [(X_valid, y_valid)],
          # eval_set =[(X_test, y_test)],
          early_stopping_rounds=8,
          verbose=0,
          # eval_metric="logloss"
          )
    run_time = time() - start_time

  else:
    start_time = time()
    m.fit(X_train, y_train)
    run_time = time() - start_time
    
  # Evaluate model
  accuracy = np.mean(m.predict(X_test) == y_test)

  start_test = time()
  y_test_pred = m.predict_proba(X_test)
  test_time = time() - start_test

  print(n)
  metric_scrores = log_binary_classification_metrics(y_test, y_test_pred)
  # pickle_and_send_artifact((y_test, y_test_pred), 'test_predictions.pkl')
  for metric_name, score in metric_scrores.items():
    print('{} :{}'.format(metric_name, score))

  df_results.loc[n] = [
                       metric_scrores.get("f1_score"),
                       metric_scrores.get("precision"), 
                       metric_scrores.get("recall"), 
                       metric_scrores.get("accuracy"),
                       accuracy,
                       run_time,
                       test_time]    
  print('-'*50)
  del m
    
train_time = timer() - start
print('The time is {:.4f} seconds'.format(train_time))

LogisticRegression
accuracy :1.0
precision :1.0
recall :1.0
f1_score :1.0
f2_score :1.0
matthews_corrcoef :1.0
cohen_kappa :1.0
true_positive_rate :1.0
true_negative_rate :1.0
positive_predictive_value :1.0
negative_predictive_value :1.0
false_positive_rate :0.0
false_negative_rate :0.0
false_discovery_rate :0.0
--------------------------------------------------
DecisionTreeClassifier
accuracy :0.9863481228668942
precision :0.9710144927536232
recall :0.9710144927536232
f1_score :0.9710144927536232
f2_score :0.9710144927536233
matthews_corrcoef :0.9620859213250518
cohen_kappa :0.9620859213250518
true_positive_rate :0.9710144927536232
true_negative_rate :0.9910714285714286
positive_predictive_value :0.9710144927536232
negative_predictive_value :0.9910714285714286
false_positive_rate :0.008928571428571428
false_negative_rate :0.028985507246376812
false_discovery_rate :0.028985507246376812
--------------------------------------------------
RandomForestClassifier
accuracy :1.0
precision :1.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/loc

accuracy :1.0
precision :1.0
recall :1.0
f1_score :1.0
f2_score :1.0
matthews_corrcoef :1.0
cohen_kappa :1.0
true_positive_rate :1.0
true_negative_rate :1.0
positive_predictive_value :1.0
negative_predictive_value :1.0
false_positive_rate :0.0
false_negative_rate :0.0
false_discovery_rate :0.0
--------------------------------------------------
GradientBoostingClassifier
accuracy :1.0
precision :1.0
recall :1.0
f1_score :1.0
f2_score :1.0
matthews_corrcoef :1.0
cohen_kappa :1.0
true_positive_rate :1.0
true_negative_rate :1.0
positive_predictive_value :1.0
negative_predictive_value :1.0
false_positive_rate :0.0
false_negative_rate :0.0
false_discovery_rate :0.0
--------------------------------------------------
HistGradientBoostingClassifier
accuracy :1.0
precision :1.0
recall :1.0
f1_score :1.0
f2_score :1.0
matthews_corrcoef :1.0
cohen_kappa :1.0
true_positive_rate :1.0
true_negative_rate :1.0
positive_predictive_value :1.0
negative_predictive_value :1.0
false_positive_rate :0.0
false_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:144: RuntimeWarning: invalid value encountered in long_scalars
/usr/loc

accuracy :0.9880546075085325
precision :0.9781021897810219
recall :0.9710144927536232
f1_score :0.9745454545454545
f2_score :0.9724238026124818
matthews_corrcoef :0.9667527105801883
cohen_kappa :0.9667417989589583
true_positive_rate :0.9710144927536232
true_negative_rate :0.9933035714285714
positive_predictive_value :0.9781021897810219
negative_predictive_value :0.9910913140311804
false_positive_rate :0.006696428571428571
false_negative_rate :0.028985507246376812
false_discovery_rate :0.021897810218978103
--------------------------------------------------
LGBMClassifier
accuracy :1.0
precision :1.0
recall :1.0
f1_score :1.0
f2_score :1.0
matthews_corrcoef :1.0
cohen_kappa :1.0
true_positive_rate :1.0
true_negative_rate :1.0
positive_predictive_value :1.0
negative_predictive_value :1.0
false_positive_rate :0.0
false_negative_rate :0.0
false_discovery_rate :0.0
--------------------------------------------------
CatBoostClassifier
accuracy :1.0
precision :1.0
recall :1.0
f1_score :1.0
f2_

In [ ]:
## Testing LinearSVC
start = timer()
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# svc = make_pipeline(StandardScaler(),
#                     LinearSVC(probability=True, random_state=0, tol=1e-5))
# svc = LinearSVC()
svc = SVC(kernel='linear', C=10)

start_time = time()
svc.fit(X_train, y_train)
run_time = time() - start_time

preds = svc.predict(X_test)
# y_test_pred = svc.predict_proba(X_test)
accuracy = np.mean(preds == y_test)


# df_results.loc['LinearSVC'] = [score, run_time]

print("LinearSVC")
metric_scrores = log_binary_classification_metrics(y_test, preds)
# pickle_and_send_artifact((y_test, y_test_pred), 'test_predictions.pkl')
# for metric_name, score in metric_scrores.items():
  # print('{} :{}'.format(metric_name, score))

df_results.loc['LinearSVC'] = [
                               metric_scrores.get("f1_score"),
                               metric_scrores.get("precision"), 
                               metric_scrores.get("recall"), 
                               metric_scrores.get("accuracy"),
                               accuracy,
                               run_time,
                               0]      
print('-'*50)
del svc

train_time = timer() - start
print('The time is {:.4f} seconds'.format(train_time))

LinearSVC


IndexError: ignored

# Training results

In [ ]:
df_results

## Save performances result to SVC file


In [ ]:
# save the training results
df_results.to_csv('performances_shallow_algorithm_lab_220412.csv')

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html#sklearn.metrics.ConfusionMatrixDisplay.from_estimator